# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fab250e3d30>
├── 'a' --> tensor([[ 1.2261, -0.4379,  1.0956],
│                   [-0.7337,  0.8397, -0.5712]])
└── 'x' --> <FastTreeValue 0x7fab250e3040>
    └── 'c' --> tensor([[ 0.0143,  0.0952,  1.6115,  1.4137],
                        [-1.1223,  0.1957, -0.2791, -0.0747],
                        [ 0.5248,  0.0435, -0.0254, -2.0615]])

In [4]:
t.a

tensor([[ 1.2261, -0.4379,  1.0956],
        [-0.7337,  0.8397, -0.5712]])

In [5]:
%timeit t.a

69.7 ns ± 0.0318 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fab250e3d30>
├── 'a' --> tensor([[ 0.8521,  0.3607,  0.4238],
│                   [-1.2237, -0.0999, -1.1530]])
└── 'x' --> <FastTreeValue 0x7fab250e3040>
    └── 'c' --> tensor([[ 0.0143,  0.0952,  1.6115,  1.4137],
                        [-1.1223,  0.1957, -0.2791, -0.0747],
                        [ 0.5248,  0.0435, -0.0254, -2.0615]])

In [7]:
%timeit t.a = new_value

69.1 ns ± 0.0645 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.2261, -0.4379,  1.0956],
               [-0.7337,  0.8397, -0.5712]]),
    x: Batch(
           c: tensor([[ 0.0143,  0.0952,  1.6115,  1.4137],
                      [-1.1223,  0.1957, -0.2791, -0.0747],
                      [ 0.5248,  0.0435, -0.0254, -2.0615]]),
       ),
)

In [10]:
b.a

tensor([[ 1.2261, -0.4379,  1.0956],
        [-0.7337,  0.8397, -0.5712]])

In [11]:
%timeit b.a

57.8 ns ± 0.0709 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.7730,  2.0081,  1.6305],
               [-0.1318, -0.5292, -1.6034]]),
    x: Batch(
           c: tensor([[ 0.0143,  0.0952,  1.6115,  1.4137],
                      [-1.1223,  0.1957, -0.2791, -0.0747],
                      [ 0.5248,  0.0435, -0.0254, -2.0615]]),
       ),
)

In [13]:
%timeit b.a = new_value

489 ns ± 0.0962 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

810 ns ± 0.184 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.9 µs ± 23 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

146 µs ± 743 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

144 µs ± 665 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fab250f8f70>
├── 'a' --> tensor([[[ 1.2261, -0.4379,  1.0956],
│                    [-0.7337,  0.8397, -0.5712]],
│           
│                   [[ 1.2261, -0.4379,  1.0956],
│                    [-0.7337,  0.8397, -0.5712]],
│           
│                   [[ 1.2261, -0.4379,  1.0956],
│                    [-0.7337,  0.8397, -0.5712]],
│           
│                   [[ 1.2261, -0.4379,  1.0956],
│                    [-0.7337,  0.8397, -0.5712]],
│           
│                   [[ 1.2261, -0.4379,  1.0956],
│                    [-0.7337,  0.8397, -0.5712]],
│           
│                   [[ 1.2261, -0.4379,  1.0956],
│                    [-0.7337,  0.8397, -0.5712]],
│           
│                   [[ 1.2261, -0.4379,  1.0956],
│                    [-0.7337,  0.8397, -0.5712]],
│           
│                   [[ 1.2261, -0.4379,  1.0956],
│                    [-0.7337,  0.8397, -0.5712]]])
└── 'x' --> <FastTreeValue 0x7faa5d311e20>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.6 µs ± 88.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7faa4d1e2d90>
├── 'a' --> tensor([[ 1.2261, -0.4379,  1.0956],
│                   [-0.7337,  0.8397, -0.5712],
│                   [ 1.2261, -0.4379,  1.0956],
│                   [-0.7337,  0.8397, -0.5712],
│                   [ 1.2261, -0.4379,  1.0956],
│                   [-0.7337,  0.8397, -0.5712],
│                   [ 1.2261, -0.4379,  1.0956],
│                   [-0.7337,  0.8397, -0.5712],
│                   [ 1.2261, -0.4379,  1.0956],
│                   [-0.7337,  0.8397, -0.5712],
│                   [ 1.2261, -0.4379,  1.0956],
│                   [-0.7337,  0.8397, -0.5712],
│                   [ 1.2261, -0.4379,  1.0956],
│                   [-0.7337,  0.8397, -0.5712],
│                   [ 1.2261, -0.4379,  1.0956],
│                   [-0.7337,  0.8397, -0.5712]])
└── 'x' --> <FastTreeValue 0x7fab2552c3a0>
    └── 'c' --> tensor([[ 0.0143,  0.0952,  1.6115,  1.4137],
                        [-1.1223,  0.1957, -0.2791, -0.0747],
                 

In [23]:
%timeit t_cat(trees)

31.4 µs ± 66.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

68.9 µs ± 87.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.2261, -0.4379,  1.0956],
                [-0.7337,  0.8397, -0.5712]],
       
               [[ 1.2261, -0.4379,  1.0956],
                [-0.7337,  0.8397, -0.5712]],
       
               [[ 1.2261, -0.4379,  1.0956],
                [-0.7337,  0.8397, -0.5712]],
       
               [[ 1.2261, -0.4379,  1.0956],
                [-0.7337,  0.8397, -0.5712]],
       
               [[ 1.2261, -0.4379,  1.0956],
                [-0.7337,  0.8397, -0.5712]],
       
               [[ 1.2261, -0.4379,  1.0956],
                [-0.7337,  0.8397, -0.5712]],
       
               [[ 1.2261, -0.4379,  1.0956],
                [-0.7337,  0.8397, -0.5712]],
       
               [[ 1.2261, -0.4379,  1.0956],
                [-0.7337,  0.8397, -0.5712]]]),
    x: Batch(
           c: tensor([[[ 0.0143,  0.0952,  1.6115,  1.4137],
                       [-1.1223,  0.1957, -0.2791, -0.0747],
                       [ 0.5248,  0.0435, -0.0254, -2.0615]],
         

In [26]:
%timeit Batch.stack(batches)

78.6 µs ± 118 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.2261, -0.4379,  1.0956],
               [-0.7337,  0.8397, -0.5712],
               [ 1.2261, -0.4379,  1.0956],
               [-0.7337,  0.8397, -0.5712],
               [ 1.2261, -0.4379,  1.0956],
               [-0.7337,  0.8397, -0.5712],
               [ 1.2261, -0.4379,  1.0956],
               [-0.7337,  0.8397, -0.5712],
               [ 1.2261, -0.4379,  1.0956],
               [-0.7337,  0.8397, -0.5712],
               [ 1.2261, -0.4379,  1.0956],
               [-0.7337,  0.8397, -0.5712],
               [ 1.2261, -0.4379,  1.0956],
               [-0.7337,  0.8397, -0.5712],
               [ 1.2261, -0.4379,  1.0956],
               [-0.7337,  0.8397, -0.5712]]),
    x: Batch(
           c: tensor([[ 0.0143,  0.0952,  1.6115,  1.4137],
                      [-1.1223,  0.1957, -0.2791, -0.0747],
                      [ 0.5248,  0.0435, -0.0254, -2.0615],
                      [ 0.0143,  0.0952,  1.6115,  1.4137],
                      [-1.1223,  

In [28]:
%timeit Batch.cat(batches)

148 µs ± 285 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

335 µs ± 1.17 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
